## Домашнее задание №1

### Предлагается реализовать поисковую систему, способную работать с proximity-оператором в запросе

#### Ограничения:
- В запросах встречаются только операторы OW/N и UW/N, где N - кол-во токенов, которое учитывается операторами
- Система должна уметь обрабатывать:
  - однотокенные запросы (без proximity-оператора), например: "молоко"
  - двухтокенные запросы с proximity-оператором между токенами, например: "молочный OW/1 ломтик"
- Система должна возвращать в ответе на запрос максимум 20 товаров
- Весь код должен быть абсолютно воспроизводимым - в режиме "Run all"

#### Цель - получить работающую поисковую систему с метриками качества на валидационном наборе запросов (прилагается к заданию):
- precision > 0.3
- recall > 0.4

#### Разбалловка за каждый пункт указана ниже, максимальное количество баллов за задание - 10

In [ ]:
from typing import Optional
import string
from dataclasses import dataclass

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

from tqdm import tqdm

np.random.seed(42)


### Дан корпус документов - база товаров с их именами

In [ ]:
dataset = pd.read_parquet("products_with_names.parquet")

In [ ]:
documents_dict = {
    doc[1]["product_id"]: doc[1]["name"] for doc in dataset.iterrows()
}

In [ ]:
@dataclass
class Document:
    doc_id: int
    name: str


documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in dataset.iterrows()]


### Пример реализации класса обработчика текста

Не обязательно использовать его в таком виде, можно реализовать любую обработку

In [ ]:
class TextProcessor:
    def __init__(self):
        self.symbols_to_replace = {"ё": "е"}
        self.stopwords = set(stopwords.words("russian"))
        self.linguist = MorphAnalyzer()

    def lowercase_text(self, text: str) -> str:
        # TODO: add some code here
        pass

    def replace_symbols(self, text: str) -> str:
        # TODO: add some code here
        pass

    def process_punctuation_simple(self, text: str) -> str:
        # TODO: add some code here
        pass

    def tokenize_simple(self, text: str) -> list[str]:
        # TODO: add some code here
        pass

    def remove_stopwords(self, tokenized_text: list[str]) -> list[str]:
        # TODO: add some code here
        pass

    def lemmatize_token(self, token: str) -> str:
        # TODO: add some code here
        pass
    
    def lemmatize_tokenized_text(self, tokenized_text: list[str]) -> list[str]:
        # TODO: add some code here
        pass

    def process_text(self, text: str) -> list[str]:
        text = self.lowercase_text(text)
        text = self.replace_symbols(text)
        text = self.process_punctuation_simple(text)
        text_tokens = self.tokenize_simple(text)
        text_tokens = self.remove_stopwords(text_tokens)
        return self.lemmatize_tokenized_text(text_tokens)
        

In [ ]:
text_processor = TextProcessor()

In [ ]:
documents_processed = [(document.doc_id, text_processor.process_text(document.name)) for document in tqdm(documents)]

### (2 балла) Составляем positional inverted index

In [ ]:
def create_positional_inverted_index(
    documents_processed: list[list[str]]
) -> dict[str, list[tuple[int, list[int]]]]:
    # TODO: add some code here
    pass


In [ ]:
positional_inverted_index = create_positional_inverted_index(documents_processed)

### (3 балла) Реализация слияния списков, учитывающего позиции токенов в документах и proximity-оператор в запросе, и функции обработки запроса

In [ ]:
def merge_posting_lists_with_condition(posting_lists, condition):
    # TODO: add some code here
    pass

In [ ]:
def search_over_positional_inverted_index(
    positional_inverted_index: dict[str, list[tuple[int, list[int]]]], 
    documents_dict: dict[int, str],
    query: str,
    limit: int = None,
) -> list[Document]:
    # TODO: add some code here
    pass

In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "картофель", limit=2)

In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "молочный OW/1 ломтик", limit=2)

In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "уксус UW/2 яблочный", limit=2)

### Читаем валидационный набор запросов с позитивными примерами товаров

Метрики будут считаться на нем

In [ ]:
result_proximity_validation_query_positives = pd.read_parquet("result_proximity_validation_query_positives.parquet")

In [ ]:
result_proximity_validation_query_positives_dict = {
    row[1].query: row[1].products.tolist()
    for row in result_proximity_validation_query_positives.iterrows()
}

In [ ]:
ground_truths_list: list[list[int]] = []
search_results_list: list[list[int]] = []

In [ ]:
for query, ground_truth_products in result_proximity_validation_query_positives_dict.items():
    ground_truths_list.append(ground_truth_products)
    search_results_list.append(
        search_over_positional_inverted_index(positional_inverted_index, documents_dict, query, limit=20)
    )

### (3 балла - за соответствие требованиям к метрикам) Посчитаем метрики качества системы 

In [ ]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [ ]:
def calculate_metrics(ground_truth_set, search_results_set):
    
    # True positives: items that are both in ground truth and search results
    tp = len(ground_truth_set.intersection(search_results_set))
    
    # Precision: tp / (tp + fp)
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    # Recall: tp / (tp + fn)
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    # F1-score: harmonic mean of precision and recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [ ]:
def calculate_validation_metrics(ground_truth_products_lists, search_result_products_lists):
    metrics = []
    for ground_truth, search in zip(ground_truth_products_lists, search_result_products_lists):
        metrics.append(
            calculate_metrics(set(ground_truth), set(x.doc_id for x in search))
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

In [ ]:
calculate_validation_metrics(ground_truths_list, search_results_list)

### (2 балла) Сравнить метрики с поиском по обратному индексу без учета позиций

Нужно:
- реализовать поисковый движок над обратным индексом без токенопозиций;
- посчитать метрики качества (precision, recall) на том же валидационном наборе запросов (игнорируя proximity-операторы);
- сравнить метрики с первым подходом и сделать вывод о полезности токенопозиций


In [ ]:
# TODO: add some code here